# Basics of spike train analysis

This is the first part of the course. 

**You will learn how to:** 
- Load spiking data into NumPy arrays and plot spike rasters.
- Calculate firing rates from spike trains.
- Calculate and interpret: interspike interval histograms, auto- and cross-correlograms.

Let's first import the packages we are going to use, and set up some plotting parameters.

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join

%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 6.0) # set default size of plots

## 1 - Loading and plotting spike trains

A spike train of a single neuron can be defined as a set of spike times. 

We will beging by learning how to load the spike times of a single (or multiple) neurons in a numpy arrays.

### Loading single and multiple spike trains

In [ ]:
exampleSpikesPath='example_spikes.txt' #path of spike train in working folder
exampleSpikeTimes = np.loadtxt(exampleSpikesPath) #load using the function

The file above contains the spike train of a single neuron in seconds.

In [ ]:
print(exampleSpikeTimes)

**Exercise:** Since you will often have to load of spike trains, you should really understand how to do it. This exercise should also test your basic understanding of passing arguments to functions. Try to fill `loadSpikeTrain()`:

In [ ]:
def loadSpikeTrain(pathToSpikeTrain):
    '''
    Inputs:
            pathToSpikeTrain: string
    Outputs: 
            spikeTimes: numpy array
    '''
    
    ### START CODE HERE ### (approx. 1 line)
    spikeTimes = np.loadtxt(pathToSpikeTrain)
    ### END CODE HERE ###
    
    return spikeTimes

In [ ]:
print(loadSpikeTrain('data_spike_trains/18_SP_C203.txt'))

Expected output: 
[  0.5766   2.8239   4.5523 ... 481.387  482.4371 482.4677]

**Exercise:** It is also useful to load multiple spike trains in our workspace. We would like to stack them in a list. Think of what should happen if the input is just a single spike train!

*Hint:* Make use of `np.loadtxt()` that we used above!

In [ ]:
def loadSpikeTrainsToList(pathList):
    '''
    Inputs:
            list_of_paths: list of strings
    Outputs: 
            list_of_spikes: numpy array
    '''
    
    ### START CODE HERE ### (approx. 1-3 lines)
    spikeTrainList=[np.loadtxt(spath) for spath in pathList]
    ### END CODE HERE ###
    
    return spikeTrainList

In [ ]:
mypath='data_spike_trains/'
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
print(loadSpikeTrainsToList(onlyfiles)[2])

Expected output:

[3.710000e-02 5.223000e-01 5.517000e-01 ... 4.816942e+02 4.818235e+02
 4.832769e+02]

### Raster plots for spike train visualization

An useful function for plotting spike trains is eventplot. Let's try plotting our example spikes:

In [ ]:
plt.eventplot(exampleSpikeTimes, colors='k')
plt.xlim([0,15])
plt.xlabel('Time (s)')
plt.yticks([1]);

Have you seen this plot before? Can you understand it?

It is possible to use eventplot to stack spike trains on top of one another. 

In [ ]:
mypath='data_spike_trains/'
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
allSpikeTrains=loadSpikeTrainsToList(onlyfiles)
plt.eventplot(allSpikeTrains,colors='k',linelengths=0.8)
plt.xlim([0,32]);

This took quite some time. The problem is that we first plotted the whole spike trains (up to 600 seconds), and then focused on the first 32 seconds. We can use logical indexing to shorten our spike trains just before plotting...

In [ ]:
trainsToPlot=[allSpikeTrains[ii][allSpikeTrains[ii]<32] for ii in range(0,len(allSpikeTrains))]
import time
startTime=time.time()
plt.eventplot(trainsToPlot,colors='k',linelengths=0.8)
endTime=time.time()
print('Short spike trains took ' + str(endTime-startTime) + ' seconds')

startTime=time.time()
plt.eventplot(allSpikeTrains,colors='k',linelengths=0.8)
plt.xlim([0,32])
endTime=time.time();
print('Long spike trains took ' + str(endTime-startTime) + ' seconds')

**Exercise:** Let's test the basics. Can you plot the spike trains of five neurons from ```allSpikeTrains```? Specifically, 1st, 12th, 13th, 14th and 19th neurons...

*Hint:* To match the raster above, focus on looking only at the first 32 seconds. You can use either of the two methods we showed!

In [ ]:
Idx=[0, 11,12,13,18]
newList=[allSpikeTrains[i][allSpikeTrains[i]<32] for i in Idx]
plt.eventplot(newList,linelengths=0.8,colors='k');

## 2 - Firing rate estimation by binning spike trains

Spike trains almost never contain the same number of events. This fact makes their manipulation for plotting and analysis harder.

### Binning spike trains

Let's first create our bins:

In [ ]:
Tmin=0 # min time in seconds
Tmax=16 # max time in seconds
dt=0.01 # time bin in seconds
binedges=np.arange(Tmin,Tmax,dt)
bincenters=binedges[:-1]+dt/2 # get the centers of the bins

And bin them using ```np.histogram```:

In [ ]:
frate,_= np.histogram(exampleSpikeTimes,binedges) # do binning
frate = frate/dt # transform counts to rates
# plot results
plt.plot(bincenters,frate)
plt.xlim([0,Tmax])
plt.xlabel('Time (s)')
plt.ylabel('Firing rate (Hz)');

**Exercise:** Let's put what we have learned in the following function

In [ ]:
def calculateFiringRate(spikeTimes,dt):
    '''
    Inputs:
            spikeTimes:
            dt:
    Outputs: 
            frate: in Hz
            bincenters: in s
    '''
    
    ### START CODE HERE ### (approx. 5 lines)
    Tmax=np.max(spikeTimes)
    Tmin=0
    binedges=np.arange(Tmin,Tmax,dt)
    bincenters=binedges[:-1]+dt/2 # get the centers of the bins
    frate,_= np.histogram(spikeTimes,binedges) # do binning
    frate = frate/dt # transform counts to rates
    ### END CODE HERE ###
    
    return frate, bincenters

In [ ]:
tshowmax=10
frate10,centers10=calculateFiringRate(exampleSpikeTimes[exampleSpikeTimes<tshowmax],0.01)
frate50,centers50=calculateFiringRate(exampleSpikeTimes[exampleSpikeTimes<tshowmax],0.05)
plt.plot(centers10,frate10,centers50,frate50)
plt.xlim([0,tshowmax])
plt.legend(('Bin 10 ms','Bin 50 ms'));

Above, we plotted the firing rate estimated with two different bin sizes. Can you understand this?

## 3 - Spike train statistics

By looking at statistical properties of spike trains, we can extract useful information.

### Inter-spike interval histogram

Below, you see the inter-spike interval histogram of a single neuron:

In [ ]:
plt.hist(np.diff(exampleSpikeTimes)*1e3,np.linspace(0,30,100))
plt.xlabel('ISI duration (ms)')
plt.ylabel('# ISIs')
plt.title('Inter-spike interval (ISI) histogram');

**Exercise:** Estimate the % of intervals that are below 2 ms by filling in `percentIntervals(spikeTimes, timeInterval)`

In [ ]:
def percentIntervals(spikeTimes,timeInterval):
    '''
    Inputs:
            spikeTimes:
            timeInterval:
    Outputs: 
            pint: percentage of ISIs below timeInterval
    '''
    
    ### START CODE HERE ### (approx. 2 lines)
    spdiffs=np.diff(spikeTimes)*1e3;
    pint=np.sum(spdiffs<timeInterval)/np.size(spdiffs)
    ### END CODE HERE ###
    
    return pint

In [ ]:
print("ISIs below 2 ms are " + str(100*percentIntervals(exampleSpikeTimes,2)) + "%")

**Expected output:**
1.15 %

### Autocorrelogram

In order to construct an auto-correlogram, we have to bin our spike times first. Since we are looking at individual spikes, one has to use a time resolution of <1 ms. Furthermore, we will look only at timescales relevant for spiking.

In [ ]:
t_res=0.5e-3
Tlag=0.04
Nlag=np.round(Tlag/t_res)
bins=np.arange(0,600,t_res)
binnedspikes,binbin=np.histogram(exampleSpikeTimes, bins)

Because NumPy's functions are not the best for our purpose, we will import an external library

In [ ]:
import pycorrelate as pyc

Now we can use the function ```pyc.ucorrelate()``` to calculate the autocorrelogram

In [ ]:
ac=pyc.ucorrelate(binnedspikes,binnedspikes,Nlag)
ytoplot=np.hstack((np.flip(ac[1:]),0,ac[1:]))/ac[0] # normalized by maximum
xtoplot=np.arange(1,Nlag)*t_res*1e3;
xtoplot=np.hstack((-np.flip(xtoplot),0,xtoplot))
plt.plot(xtoplot,ytoplot)
#Setup plot appearance
plt.title('Autocorrelogram')
plt.xlabel('Time lag (ms)')
plt.ylabel('Correlation (norm.)')
plt.xlim((-Tlag*1e3,Tlag*1e3));

In order to calculate the autocorrelogram, we had to call multiple functions. Often, it helps to define a new function that can do the calculation for us.

**Exercise:** Make the autocorrelogram calculation into a function! 
Bonus: think of how to add an extra option that plots the result as well...

In [ ]:
def calculateAutoCorrelogram(spiketrain,timeResolution,timeLag):
    '''
    Inputs:
            spiketrain: numpy array of spike times in seconds
            timeResolution: resolution in seconds
            timeLag: number of seconds of time lag
    Outputs: 
            xvals: numpy array of times
            yvals: numpy array of autocorrelogram values
    '''
    ### START CODE HERE ### (approx. 5-10 lines)
    Nlag=np.round(timeLag/timeResolution)
    bins=np.arange(0,np.max(spiketrain),timeResolution)
    binnedspikes,binbin=np.histogram(spiketrain, bins)
    ac=pyc.ucorrelate(binnedspikes,binnedspikes,Nlag)
    yvals=np.hstack((np.flip(ac[1:]),0,ac[1:]))/ac[0]
    xvals=np.arange(1,Nlag)*timeResolution;
    xvals=np.hstack((-np.flip(xvals),0,xvals))
    ### END CODE HERE ###
    return yvals, xvals

In [ ]:
py,px=calculateAutoCorrelogram(allSpikeTrains[2],1e-3,40e-3)
print(py)
plt.plot(px,py)
plt.title('Autocorrelogram')
plt.xlabel('Time lag (s)')
plt.ylabel('Correlation (norm.)');

**Expected output:** [0.05465071 0.06013122 0.0547279 ... 0.0547279  0.06013122 0.05465071]

### Crosscorrelogram

We can also correlate the spike trains of two different neurons. The resulting function is called the crosscorrelogram. By examing the crosscorrelogram, we can infer useful information.

We will now fill in the following function that calculates the crosscorrelogram between spike_train1 and spike_train2. It should be similar as the autocorrelogram above.

In [ ]:
def calculateCrossCorrelogram(spiketrain1,spiketrain2,timeResolution,timeLag):
    ### START CODE HERE ### (approx. 2 lines)
    Nlag=np.round(timeLag/timeResolution)
    bins=np.arange(0,np.max((np.max(spiketrain1),np.max(spiketrain2))),timeResolution)
    binnedspikes1,bb=np.histogram(spiketrain1, bins)
    binnedspikes2,bb=np.histogram(spiketrain2, bins)
    ac1=pyc.ucorrelate(binnedspikes1,binnedspikes2,Nlag)
    ac2=pyc.ucorrelate(binnedspikes2,binnedspikes1,Nlag)
    yvals=np.hstack((np.flip(ac2[1:]),ac1))
    
    xvals=np.arange(1,Nlag)*timeResolution;
    xvals=np.hstack((-np.flip(xvals),0,xvals))
    ### END CODE HERE ###
    return yvals,xvals

In [ ]:
py,px=calculateCrossCorrelogram(allSpikeTrains[0],allSpikeTrains[2],0.5e-3,60e-3)# pairs(0,2),
plt.plot(px,py)
plt.ylim([0,np.max(py)]);

Expected output: 

Can we understand the cross-correlogram by looking at the spike trains?

In [ ]:
Idx=[0, 2]
newList=[allSpikeTrains[i][allSpikeTrains[i]<32] for i in Idx]
plt.eventplot(newList,linelengths=0.8);

**Exercise:** Using the function from above, calculate and plot the cross-correlogram between the 3rd and 4th spike trains of ```allSpikeTrains```. How do you interpret what you see?

In [ ]:
### START CODE HERE ### (approx. 2 lines)

### END CODE HERE ###

Well done!